In [ ]:
import pandas as pd
label = pd.read_csv("/home/iatell/projects/meta-learning/data/seq_line_labels.csv")
label["seq_len"] = label["endIndex"] - label["startIndex"]
label

# model


In [33]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F


def mdn_split_params(raw_params, n_components):
    """
    raw_params: (B, 3K) tensor from mdn_head
    returns:
        pi    (B, K) mixture weights
        mu    (B, K) means
        sigma (B, K) std devs
    """
    B, threeK = raw_params.shape
    assert threeK == 3 * n_components

    raw = raw_params.view(B, n_components, 3)

    pi = raw[..., 0]                 # (B,K)
    mu = raw[..., 1]                 # (B,K)
    sigma = raw[..., 2]              # (B,K)

    pi = F.softmax(pi, dim=-1)       # weights sum to 1
    sigma = F.softplus(sigma) + 1e-4 # strictly positive
    return pi, mu, sigma


def mdn_nll_multitarget(y_line, pi, mu, sigma):
    """
    Negative log-likelihood for MDN with multiple valid targets per sample.
    Args:
        y_line : (B, L) padded targets (0 where invalid)
        pi, mu, sigma : (B, K) MDN params
    Returns:
        scalar loss
    """
    B, K = mu.shape
    losses = []

    for b in range(B):
        valid_y = y_line[b][y_line[b] > 0]  # (M,)
        if len(valid_y) == 0:
            continue

        # expand to (M, K)
        y_exp = valid_y.unsqueeze(-1).expand(-1, K)

        log_prob = -0.5 * ((y_exp - mu[b]) / (sigma[b] + 1e-8))**2 \
                   - torch.log(sigma[b] + 1e-8) \
                   - 0.5 * torch.log(torch.tensor(2.0 * torch.pi, device=y_line.device))

        log_mix = torch.log(pi[b] + 1e-8) + log_prob
        log_sum = torch.logsumexp(log_mix, dim=-1)  # (M,)

        losses.append(-log_sum.mean())

    if len(losses) == 0:
        return torch.tensor(0.0, device=y_line.device, requires_grad=True)

    return torch.stack(losses).mean()


class CNNLSTM_MDN(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim=128, num_layers=1,
                 lr=1e-3, n_components=5, cnn_channels=64, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()

        # CNN
        self.conv1 = nn.Conv1d(input_dim, cnn_channels, kernel_size=1)
        self.conv3 = nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(cnn_channels)
        self.bn3   = nn.BatchNorm1d(cnn_channels)

        # LSTM
        fused_dim = 2 * cnn_channels
        self.lstm = nn.LSTM(fused_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # MDN output (single step)
        self.mdn_head = nn.Linear(hidden_dim, 3 * n_components)
        self.n_components = n_components
        self.lr = lr

    def forward(self, X, lengths=None):
        # CNN
        x = X.transpose(1, 2)
        x1 = F.relu(self.bn1(self.conv1(x)))
        x3 = F.relu(self.bn3(self.conv3(x)))
        xf = torch.cat([x1, x3], dim=1).transpose(1, 2)

        # LSTM
        _, (h_last, _) = self.lstm(xf)
        last_h = h_last[-1]  # (B,H)

        # MDN (one step)
        raw = self.mdn_head(last_h)  # (B,3K)
        pi, mu, sigma = mdn_split_params(raw, self.n_components)
        return {"pi": pi, "mu": mu, "sigma": sigma}

    def training_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y_line, lengths = batch
        mdn = self(X, lengths)
        loss = mdn_nll_multitarget(y_line, mdn["pi"], mdn["mu"], mdn["sigma"])
        self.log("val/loss", loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


# train

In [ ]:
import sys
from pathlib import Path

# Current notebook location
notebook_path = Path().resolve()

# Add parent folder (meta/) to sys.path
sys.path.append(str(notebook_path.parent))
import joblib
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix,mean_absolute_error,mean_squared_error
from datetime import datetime
from preprocess.multi_regression_seq_dif import preprocess_sequences_csv_multilines
# from models.LSTM.lstm_multi_line_reg_seq_dif import LSTMMultiRegressor
from utils.print_batch import print_batch
from utils.to_address import to_address
from utils.json_to_csv import json_to_csv_in_memory
from utils.padding_batch_reg import collate_batch
import pandas as pd
import io
import numpy as np
import os

from sklearn.metrics import accuracy_score, f1_score
# ---------------- Evaluation ---------------- #
@torch.no_grad()
def evaluate_model_mdn(model, val_loader, zero_idx=0, threshold=0.7):
    """
    Evaluate CNN–LSTM–MDN model (last-output version).

    Args
    ----
    model : pl.LightningModule with MDN forward
    val_loader : DataLoader yielding (X, y, lengths)
    zero_idx : which mixture component is considered "no-line" (usually 0)
    threshold : if pi[:,zero_idx] > threshold → predict invalid

    Returns
    -------
    dict with mse, mae, acc, f1
    """
    model.eval()
    all_preds_reg, all_labels_reg = [], []
    all_preds_len, all_labels_len = [], []

    device = next(model.parameters()).device

    with torch.no_grad():
        for X_batch, y_batch, lengths in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            mdn = model(X_batch, lengths)
            pi, mu, sigma = mdn["pi"], mdn["mu"], mdn["sigma"]  # (B,K)

            # regression expectation
            y_pred = (pi * mu).sum(dim=-1)  # (B,)
            B = y_batch.size(0)
            y_len = (y_batch > 0).sum(dim=1)                # (B,)
            idx = torch.clamp(y_len - 1, min=0)             # last valid index
            y_true = y_batch[torch.arange(B, device=y_batch.device), idx]  # (B,)
            # only last step
            # print("lengths(features):", lengths[:10])
            # print("lengths(labels):", y_len[:10])

            all_preds_reg.append(y_pred.cpu().numpy())
            all_labels_reg.append(y_true.cpu().numpy())

            # validity classification
            pi_zero = pi[:, zero_idx]  # (B,)
            pred_valid = (pi_zero < (1 - threshold)).long()
            true_valid = torch.ones_like(pred_valid)  # last step always valid

            all_preds_len.extend(pred_valid.cpu().numpy().tolist())
            all_labels_len.extend(true_valid.cpu().numpy().tolist())


        # ----- Regression metrics -----
    all_preds_reg = np.concatenate(all_preds_reg)  # (N,)
    all_labels_reg = np.concatenate(all_labels_reg)
    mse = ((all_preds_reg - all_labels_reg) ** 2).mean()
    mae = np.abs(all_preds_reg - all_labels_reg).mean()
    # ----- Validity metrics -----
    acc = accuracy_score(all_labels_len, all_preds_len)
    f1 = f1_score(all_labels_len, all_preds_len, average="macro")

    print("\n📊 Validation Metrics (MDN, last-output):")
    print(f"  Regression → MSE: {mse:.6f}, MAE: {mae:.6f}")
    print(f"  Validity   → Acc: {acc:.4f}, F1: {f1:.4f}")

    return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
# ---------------- Train ---------------- #
def train_model(
    data_csv,
    labels_csv,
    model_out_dir="models/saved_models",
    do_validation=True,
    hidden_dim=128,
    num_layers=1,
    lr=0.001,
    batch_size=32,
    max_epochs=400,
    save_model=False,
    return_val_accuracy = True
):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_out = f"{model_out_dir}/lstm_model_multireg_{timestamp}.pt"
    meta_out  = f"{model_out_dir}/lstm_meta_multireg_{timestamp}.pkl"

    # Preprocess: pad linePrices and sequences
    if do_validation:
        train_ds, val_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=True,
            for_xgboost=False,
            debug_sample=True
        )
    else:
        train_ds, df, feature_cols, max_len_y = preprocess_sequences_csv_multilines(
            data_csv, labels_csv,
            val_split=False,
            for_xgboost=False,
            debug_sample=False
        )
        val_ds = None

    sample = train_ds[0][0]  # first sample's features
    if isinstance(sample, dict):  # multiple feature groups
        input_dim = sample['main'].shape[1]
    else:  # single tensor
        input_dim = sample.shape[1]

    model = CNNLSTM_MDN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        lr=lr
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_batch) if val_ds else None

    trainer = pl.Trainer(max_epochs=max_epochs, accelerator="auto", devices=1)
    trainer.fit(model, train_loader, val_loader)

    if save_model:
        os.makedirs(model_out_dir, exist_ok=True)
        trainer.save_checkpoint(model_out)
        joblib.dump({
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers,
            "max_len_y": max_len_y,
            "feature_cols": feature_cols
        }, meta_out)
        print(f"✅ Model saved to {model_out}")
        print(f"✅ Meta saved to {meta_out}")
    # --- Evaluation --- #
    if do_validation:
        mse, mae, acc, f1 = evaluate_model_mdn(model, val_loader)
        if return_val_accuracy:
            return {"mse": mse, "mae": mae, "acc": acc, "f1": f1}
        
if __name__ == "__main__":
    train_model(
        "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv",
        "/home/iatell/projects/meta-learning/data/seq_line_labels.csv",
        save_model=True,
        do_validation=True
    )


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type        | Params | Mode 
-------------------------------------------------
0 | conv1    | Conv1d      | 832    | train
1 | conv3    | Conv1d      | 2.4 K  | train
2 | bn1      | BatchNorm1d | 128    | train
3 | bn3      | BatchNorm1d | 128    | train
4 | lstm     | LSTM        | 132 K  | train
5 | mdn_head | Linear      | 1.9 K  | train
-------------------------------------------------
137 K     Trainable params
0         Non-trainable params
137 K     Total params
0.550     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode



=== DEBUG SAMPLE CHECK ===

--- Sequence 0 ---
Label (linePrices padded): [0.774234 0.941543 1.004005 0.       0.       0.      ]
Shape: (56, 12)
First few rows:
 [[ 1.37079102e+04  1.38185498e+04  1.27500000e+04  1.33800000e+04
   8.60715625e+03  7.60030076e-02 -2.25254402e-01  4.32772011e-01
   1.00000000e+00  3.37409645e-01  1.92125893e+00  1.75619051e-01]
 [ 1.33821602e+04  1.54734902e+04  1.28900195e+04  1.46751104e+04
   2.00781660e+04  5.40071189e-01  8.74627411e-01  3.32912445e-01
   2.00000000e+00  6.17487133e-01  3.80633444e-01  1.62226200e+00]
 [ 1.46900000e+04  1.53075596e+04  1.41500000e+04  1.49195098e+04
   1.59054824e+04  2.63643831e-01  1.55930668e-01  3.66879702e-01
   2.00000000e+00  1.69077599e+00  2.35283875e+00  7.18611121e-01]
 [ 1.49195098e+04  1.52800000e+04  1.39180400e+04  1.50595400e+04
   2.52244141e+04  1.50006071e-01  9.52796489e-02  6.81423306e-01
   2.00000000e+00  1.57437694e+00  5.00000000e+00  2.20136404e-01]
 [ 1.50595596e+04  1.71762402e+04  1.460

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/iatell/envs/Rllib2.43/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=400` reached.


✅ Model saved to models/saved_models/lstm_model_multireg_20250903_223155.pt
✅ Meta saved to models/saved_models/lstm_meta_multireg_20250903_223155.pkl

📊 Validation Metrics (MDN, last-output):
  Regression → MSE: 0.042862, MAE: 0.167279
  Validity   → Acc: 1.0000, F1: 1.0000


# server

In [28]:
# server_cnn_mdn.py
import glob
import joblib
import torch
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
# from models.LSTM.cnn_lstm_mdn import CNNLSTM_MDN  # <-- your updated "last-output" model

app = Flask(__name__)

# ---------------- Load model and meta ----------------
meta_path = glob.glob("models/saved_models/lstm_meta_multireg_*.pkl")[0]
state_path = glob.glob("models/saved_models/lstm_model_multireg*.pt")[0]

meta = joblib.load(meta_path)
FEATURES = meta['feature_cols']

def one_sample(df_slice):
    return df_slice[FEATURES].values.astype(np.float32)

def load_raw_data_serve(candle_csv, label_csv):
    candles = pd.read_csv(candle_csv, parse_dates=['timestamp'])
    labels  = pd.read_csv(label_csv, header=None,
                          names=['timestamp', 'last_close', 'line_raw'])
    labels['timestamp'] = pd.to_datetime(labels['timestamp'])

    df = pd.merge(candles, labels[['timestamp', 'line_raw']], on='timestamp', how='left')
    df['label_price'] = df['line_raw'].str.extract(r"\(([^)]+)\)")[0].astype(float)
    df['has_line'] = (~df['label_price'].isna()).astype(int)
    df['log_factor'] = np.where(df['has_line'],
                                np.log(np.maximum(df['label_price'], 1e-6)),
                                np.nan)
    return df

# ---------------- Model ----------------
model = CNNLSTM_MDN(
    input_dim=meta['input_dim'],
    hidden_dim=meta.get('hidden_dim', 128),
    num_layers=meta.get('num_layers', 1),
    n_components=meta.get('n_components', 5),
    cnn_channels=meta.get('cnn_channels', 64),
    dropout=meta.get('dropout', 0.1)
)
model = CNNLSTM_MDN.load_from_checkpoint(state_path)
model.eval()

# ---------------- Load data ----------------
df = load_raw_data_serve(
    "/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles_prop.csv",
    "/home/iatell/projects/meta-learning/data/ohlcv_log(2).csv"
)

# ---------------- Routes ----------------
@app.route("/")
def home():
    return render_template("sequential.html")

@app.route("/candles")
def candles():
    dense = df.set_index('timestamp').asfreq('D').ffill()
    return jsonify([
        {'time': int(ts.timestamp()),
         'open': float(row.open),
         'high': float(row.high),
         'low': float(row.low),
         'close': float(row.close)}
        for ts, row in dense.iterrows()
    ])

@app.route("/predict", methods=['POST'])
def predict():
    data = request.get_json(force=True)
    idxs = data.get('idxs')        # list of indices
    seq_len = data.get('seq_len')  # int

    if not idxs or not isinstance(idxs, list):
        return jsonify({"error": "Provide 'idxs' as a list"}), 400
    if not seq_len or not isinstance(seq_len, int):
        return jsonify({"error": "Provide 'seq_len' as an int"}), 400

    all_pred_prices = []
    all_pred_sigmas = []
    all_pi = []

    for idx in idxs:
        seq_df = df.iloc[idx - seq_len + 1: idx + 1]
        last_close = seq_df.iloc[-1]['close']

        X_np = one_sample(seq_df)
        X_t = torch.from_numpy(X_np.astype(np.float32)).unsqueeze(0)
        with torch.no_grad():
            mdn_out = model(X_t)

        # Now outputs are (B,K) — no time dimension
        pi    = mdn_out['pi'][0].cpu().numpy()
        mu    = mdn_out['mu'][0].cpu().numpy()
        sigma = mdn_out['sigma'][0].cpu().numpy()

        all_pred_prices.append((last_close * (mu)).tolist())
        all_pred_sigmas.append((last_close * sigma).tolist())
        all_pi.append(pi.tolist())

    return jsonify({
        'pred_prices': all_pred_prices,
        'pred_sigmas': all_pred_sigmas,
        'pi': all_pi
    })

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2025 22:19:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:43] "GET /candles HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:19:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:10] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2025 22:20:10] "POST /predict HTTP/1.1" 200 -
